In [2]:
import pyodbc
import os

server = 'tcp:db'    # The hostname of the database server, format tcp:<hostname> 
database = 'TestDB'  # The name of the database to use
username = 'SA'      # The name of the database user
password = os.environ['MSSQL_SA_PASSWORD'] # The database password, here we injected it to the Jupyter
                                           # container as an environment variable in docker-compose.yml
driver = os.environ['MSSQL_DRIVER']        # The database driver to use, provided by the image

# Connect to the database server using pyodbc. Note that we have added
# TrustServerCertificate=yes to the connection string, as the Microsoft SQL
# server container is using a self-signed certificate.
conn_string = 'DRIVER={'  + driver + '};SERVER=' + server + ';DATABASE=' + database + ';TrustServerCertificate=yes;UID=' + username + ';PWD=' + password
connection = pyodbc.connect(conn_string)
cursor = connection.cursor()

# Perform a sample query to the database server to test connectivity.
# This should print something like:
# Microsoft SQL Server 2019 (RTM-CU18) (KB5017593) - 15.0.4261.1 (X64) 
#	 Sep 12 2022 15:07:06 
#	 Copyright (C) 2019 Microsoft Corporation
#	 Developer Edition (64-bit) on Linux (Ubuntu 20.04.5 LTS) <X64>
cursor.execute("SELECT @@version;") 
row = cursor.fetchone() 
while row: 
    print(row[0])
    row = cursor.fetchone()

Microsoft SQL Server 2019 (RTM-CU18) (KB5017593) - 15.0.4261.1 (X64) 
	Sep 12 2022 15:07:06 
	Copyright (C) 2019 Microsoft Corporation
	Developer Edition (64-bit) on Linux (Ubuntu 20.04.5 LTS) <X64>


In [25]:
from urllib.parse import quote_plus
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Boolean
from sqlalchemy.orm import sessionmaker

# Use the same connection string we used with pyodbc for simplicity. Note that
# sqlalchemy requires the connection string to be URL-encoded.
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % quote_plus(conn_string))

# Create an object which will be used for ORM
Base = declarative_base()
class Task(Base):
    __tablename__ = 'TodoList'
    
    id = Column(Integer, primary_key = True)
    task = Column(String)
    completed = Column(Boolean)
    
    def __repr__(self):
        return "(task = '%s', completed = '%s')" % (self.task, self.completed)
    
Base.metadata.create_all(engine)

Session = sessionmaker(bind = engine)
session = Session()
task1 = Task(task = 'Verify integration between SQL Server and Jupyter container.', completed = True)
task2 = Task(task = 'Push commits upstream.', completed = False)
session.add(task1)
session.add(task2)
session.commit()

session.query(Task).all()

[(task = 'Verify integration between SQL Server and Jupyter container.', completed = 'True'),
 (task = 'Push commits upstream.', completed = 'False')]